building the iterator is almost as hard as building the model..

In [9]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import glob

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# df['tag'] = df['tag'].map({'0': '0', 'S': '1', 'C': '2', 'G': '3', 'T': '4', 'B': '5','D': '6' })

In [10]:
# !pip3 install sklearn

In [81]:
path= '/Users/tancredicp/Desktop/sailing_app/data/ML Training Resources/'
all_files = glob.glob(path + "/*.csv")


# listOfNormalizedDatFrames = []
# listofTags = []

Data = []

counter = -1

for filename in all_files:
    temporary_dict = {}

    
    
    dfSpecific = pd.read_csv(filename, index_col=None, header=0)
    print(dfSpecific.shape)
    #I was lazy and did not check where the NaNs where but they were being annoying so made them all 0s.
    dfSpecific.fillna(0, inplace= True)
    
    dfSpecificY = dfSpecific[['tag']]
    dfSpecificY = dfSpecificY['tag'] #.astype('category')
    temporary_dict['tags'] = np.array(dfSpecificY)
    

    
    dfSpecificX = dfSpecific.drop(columns=['tag', 'tag_name', 'boat_class', 'time'])
    scaler = StandardScaler()
    columnsBeingStandardize = ['lat', 'lon', 'sog', 'heel', 'clew_load','pitch']
    dfSpecificX[columnsBeingStandardize] = scaler.fit_transform(dfSpecificX[columnsBeingStandardize])
    
    
    array1 = np.array(dfSpecificX[['lat', 'lon', 'sog', 'cog']])
    
    temporary_dict['track'] = array1
    
    Data.append(temporary_dict)
    
#     listOfNormalizedDatFrames[counter] = array1
    
#     break

# X = pd.concat(listOfNormalizedDatFrames, axis=0, ignore_index=True)
# y = pd.concat(listofTags, axis=0, ignore_index=True)

(3594, 12)
(2941, 12)
(2796, 12)
(528, 12)
(976, 12)
(11838, 12)
(7084, 12)
(5679, 12)
(3508, 12)
(7084, 12)
(2283, 12)
(2498, 12)
(2515, 12)
(2421, 12)
(3150, 12)
(2770, 12)
(3154, 12)
(3170, 12)
(2949, 12)
(3342, 12)
(2986, 12)
(3409, 12)
(3360, 12)
(2283, 12)


In [53]:
Data

[{'tags': array(['S', 'S', 'S', ..., 'C', 'C', 'C'], dtype=object),
  'track': array([[ 0.51690643, -0.51615816,  1.37163749, -0.3       ],
         [ 0.5261856 , -0.53723416,  1.51186259, -0.31111111],
         [ 0.53623804, -0.55701318,  1.45577255, -0.27222222],
         ...,
         [ 0.99555713, -1.3452558 , -1.19124586, -0.36666667],
         [ 0.99555713, -1.34752553, -1.14594237, -0.41666667],
         [ 0.99581488, -1.34979525, -1.16967277, -0.43888889]])},
 {'tags': array(['A', 'A', 'A', ..., 'C', 'C', 'C'], dtype=object),
  'track': array([[ 1.0800956 , -0.80381174, -1.69333974,  0.35555556],
         [ 1.11360578, -0.72625998, -1.66929835,  0.20555556],
         [ 1.05220063, -0.34540862,  0.13380649,  0.66666667],
         ...,
         [ 0.81400663, -1.76865612, -1.71956672, -0.47777778],
         [ 0.81455004, -1.770226  , -1.59061741, -0.31666667],
         [ 0.81473118, -1.77179587, -1.64525695, -0.41111111]])},
 {'tags': array(['C', 'C', 'C', ..., 'C', 'C', 'C'], dty

In [22]:
X.shape

(3594, 4)

In [23]:
len(all_files)

24

In [80]:
from torch.utils.data import Dataset
from torch.utils.data import Buc

# dataset = tf.data.Dataset.from_tensor_slices((images, labels))
# dataset

ImportError: cannot import name 'data' from 'torch.utils.data' (/Users/tancredicp/Desktop/sailing_env/sailing/lib/python3.7/site-packages/torch/utils/data/__init__.py)

In [75]:
class SailingData(Dataset):
    def __init__ (self, pl):
        self.samples = []
        
        for d in pl:
            self.samples.append((d['track'], d['tags']))
            
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
            return(self.samples[idx])
        
        
    
        
    

In [76]:
Datz = SailingData(Data)

In [79]:
Datz.__getitem__(0)

(array([[ 0.51690643, -0.51615816,  1.37163749, -0.3       ],
        [ 0.5261856 , -0.53723416,  1.51186259, -0.31111111],
        [ 0.53623804, -0.55701318,  1.45577255, -0.27222222],
        ...,
        [ 0.99555713, -1.3452558 , -1.19124586, -0.36666667],
        [ 0.99555713, -1.34752553, -1.14594237, -0.41666667],
        [ 0.99581488, -1.34979525, -1.16967277, -0.43888889]]),
 array(['S', 'S', 'S', ..., 'C', 'C', 'C'], dtype=object))

In [ ]:
Dat